# TP 2 : récursivité et conditions

Dans tout le TP, il est interdit d'utiliser des boucles (pas de `for` ni `while`).

Rappel : pensez à tester toutes vos fonctions sur des exemples.

## Petites questions

1. Définir une fonction `divise` telle que `divise a b` renvoie `true` si  `a` divise `b`, `false` sinon. Il est interdit d'utiliser `if`.
2. Définir une fonction récursive `somme` telle que `somme n` renvoie $\sum_{k=1}^n k^2$.
3. Définir une fonction récursive `u` telle que `u n` renvoie $u_n$ définie par :
$$u_0 = 42$$
$$u_{n} = 3\sqrt{u_{n - 1}} + 2$$
4. Définir une fonction récursive `v` telle que `v a n` renvoie $v_n$ définie par (on fera attention à n'utiliser qu'un appel récursif) :
$$v_0 = a$$
$$v_{n + 1} = \frac{1}{2}(v_n + \frac{a}{v_n})$$
5. Calculer `v a n` pour plusieurs valeurs de `a` et de `n` : que peut-on conjecturer sur la limite de $v_n$ ? Optionnel : le démontrer.  
6. Soient $a$ et $b$ deux entiers et $q, r$ le quotient et reste de la division euclidienne de $a$ par $b$ ($a = bq + r$). En utilisant le fait que $PGCD(a, b)$ = $PGCD(b, r)$, écrire une fonction `euclide` telle que `euclide a b` renvoie le PGCD de $a$ et $b$.

In [3]:
(*1*)let divise a b =
    a mod b = 0 ;;
divise 2 1

val divise : int -> int -> bool = <fun>


- : bool = true


In [23]:
(*2*)let rec somme n = 
    if n = 1. then 1.
    else n**2. +.somme(n-.1.);;
somme 2.

val somme : float -> float = <fun>


- : float = 5.


In [30]:
(*3*)let rec u n =
    if n = 0. then 42.
    else 3.*.(u(n-.1.))**0.5 +.2.;;
u 2.

val u : float -> float = <fun>


- : float = 15.8917241139108523


In [1]:
(*4*)let rec v a n =
    if n = 0. then a
    else (1./.2.)*.(v a( n-.1.)+.a/.v a(n-.1.));;
v 2. 2.;;
v 3. 5.;;
v 3. 7.;;
v 10. 20.;;
(*on peut conjecturer que quand n tend vers l'infini v(a,n) converge vers racine de a*)

val v : float -> float -> float = <fun>


- : float = 1.41666666666666652


- : float = 1.73205080756887719


- : float = 1.73205080756887719


- : float = 3.16227766016837908


interrupt: intterupt

In [6]:
(*6*)let rec euclide a b = 
    if a mod b = 0 then b
    else euclide b (a mod b);;
euclide 12 12

val euclide : int -> int -> int = <fun>


- : int = 12


## Exponentiation rapide

1. Écrire une fonction récursive `puissance` naïve (c'est à dire très simple) telle que `puissance a n` renvoie $a^n$. Combien effectue t-elle de multiplications (en fonction de n) ?
2. Écrire une fonction récursive `exp_rapide` pour calculer $a^n$, en utilisant les relations suivantes :
$$ 
\begin{cases} 
a^n = (a^{\frac{n}{2}})^2 ~~~~~~~~~\text{si }n\text{ est pair}\\
a^{n} = a \times (a^{\frac{n-1}{2}})^2 ~~~~~\text{sinon}
\end{cases}
$$
**Attention** : pour mettre au carré, utiliser une variable plutôt que faire 2 appels récursifs, ce qui augmenterait beaucoup le nombre d'opérations.  
**Remarque** : On montrera plus tard que cette 2ème version demande de l'ordre de $\ln(n)$ multiplications seulement.

In [1]:
(*1*)let rec puissance a n =
    if n = 0 then 1
    else puissance a (n-1)*a;;
puissance 3 3
(*Elle effectue n opérations*)

val puissance : int -> int -> int = <fun>


- : int = 27


In [6]:
(*2*)let rec exp_rapide a n =
    if n = 0 then 1
    else if n mod 2 = 0 then let b = exp_rapide a (n/2) in b*b
    else let c = exp_rapide a ((n-1)/2) in a*c*c;;
exp_rapide 2 5

val exp_rapide : int -> int -> int = <fun>


- : int = 32


# Accumulateur

On a vu dans le cours sur la récursivité (avec l'exemple de la suite de Fibonacci) qu'un accumulateur est un argument que l'on ajoute à une fonction pour calculer sa valeur de retour.  
1. Écrire une fonction `fact` telle que `fact acc n` renvoie `n`!, en utilisant `acc` comme accumulateur. Voici à quoi va ressemble `fact` :
```ocaml
let rec fact acc n =
    if n = 0 then acc  (* on renvoie l'accumulateur qui contient le résultat *)
    else fact ... (* appel récursif en modifiant l'accumulateur *)
```
2. En utilisant `fact` et l'application partielle de fonction, définir `f : int -> int` renvoyant la factoielle d'un entier.

**Remarque** : le but ici est juste de vous entraîner à savoir utiliser un accumulateur, qui sont parfois utiles (comme pour la fonction `fibo` du cours). En DS ou concours on évitera d'utiliser un accumulateur lorsqu'il y en a pas besoin (comme pour la fonction `fact`...), car cela rend le code plus compliqué.  

In [13]:
(*1*)let rec fact acc n =
    if n = 0 then acc
    else fact (acc*n) (n-1) in
fact 1 3

- : int = 6


In [19]:
(*2*)let ffact n = fact 1 n ;;
ffact 3

val ffact : int -> int = <fun>


- : int = 0


## Temps de vol de la suite de Syracuse

La suite de Syracuse d'un entier $a$ est définie par :  
$$u_0 = a$$
$$u_{n+1} =
\begin{cases} 
     \frac{u_n}{2}, \text{si } n \text{ est pair}\\
    3u_n + 1, \text{sinon}\\
\end{cases}$$

Le temps de vol de $(u_n)_n$ est le plus petit entier $t$ tel que $u_t = 1$.  

1. Écrire une fonction récursive `temps_vol` telle que `temps_vol a` renvoie le temps de vol de $(u_n)_n$ (où $u_0 = a$).

In [74]:
let rec temps_vol a =
    print_int(a);
    print_newline ();
    if a<= 1 then 1
    else if a mod 2 = 0 then temps_vol (a/2)
    else temps_vol (3*a+1);;
temps_vol 14

val temps_vol : int -> int = <fun>


14
7
22
11
34
17
52
26
13
40
20
10
5
16
8
4
2
1


- : int = 1


## Fonction mystérieuse

Tester la fonction suivante, conjecturer un théorème et prouvez-le par récurrence.

In [86]:
let rec f n =
    if n > 100 then n - 10 
    else f (f (n + 11)) ;;
f 0;;
f 101;;
f 102;;
(* pour tout n de 0 à 101 inclus la fonction renvoie 91 et pour tout n allant de 102 à plus l'infini 
la fonction renvoie n-10 *)

val f : int -> int = <fun>


- : int = 91


- : int = 91


- : int = 92


## Fonctions mutuellement récursives

Il est possible de définir simultanément deux fonctions `f` et `g`, dépendant l'une de l'autre avec `and` :
```ocaml
let rec f x = ... and g y = ... in ...
```

Écrire deux fonctions `u` et `v` permettant de calculer le $nième$ terme des suites définies par :
$$u_0 = 2$$
$$v_0 = 3$$
$$u_{n + 1} = u_n - u_n v_n$$
$$v_{n + 1} = v_n + u_n v_n$$

In [93]:
let rec u n =
    (if n = 0 then 2 else u ((n-1)) -(u (n-1)) * (v (n-1))) and
    v n = (if n = 0 then 3 else (v (n-1)) + (u (n-1))*(v (n-1)));;
u 10;;

val u : int -> int = <fun>
val v : int -> int = <fun>


- : int = -2424229655480369152


## Retour sur les tours de Hanoi

**Remarque** : le problème des tours de Hanoi a déjà été vu en stage, mais on le refait ici en OCaml. En outre, il est utile de réactiver sa mémoire pour se souvenir des méthodes et il m'arrivera pendant l'année de redonner des exercices déjà posés.

![](../img/hanoi.png)

$n$ disques sont posés sur la tige à gauche. L'objectif est de déplacer tous les disques sur la tige à droite :

![](../img/hanoi2.png)

Règles du jeu :

- On ne peut déplacer qu'un disque à la fois (celui tout en haut), sur une autre tige.
- Il est interdit de poser un disque sur un autre plus petit.

Exemple de premier déplacement valide :

![](../img/hanoi3.png)

On souhaite écrire une fonction récursive `hanoi` telle que `hanoi n tige1 tige2` affiche une suite de déplacements (avec des `print_int`) permettant de déplacer $n$ disques depuis `tige1` vers `tige2`. On supposera que les tiges sont numérotées 0, 1, 2 (de gauche à droite).

1. Supposons que vous sachiez déplacer $n-1$ disques d'une tige à une autre. Comment déplacer $n$ disques d'une tige à une autre ?
2. Écrire `hanoi`.

In [21]:
let rec hanoi n tige1 tige2 = 
    if n = 0 then ()
    else let k = 3 -tige1 -tige2 in
    hanoi (n-1) tige1 k;
    print_string "Déplacer ";
    print_int tige1;
    print_string " vers ";
    print_int k;
    print_newline();
    hanoi(n-1) k tige2;;
hanoi 3 0 2

val hanoi : int -> int -> int -> unit = <fun>


Déplacer 0 vers 1
Déplacer 0 vers 2
Déplacer 2 vers 0
Déplacer 0 vers 1
Déplacer 1 vers 2
Déplacer 1 vers 0
Déplacer 0 vers 1


- : unit = ()
